

Desafío 1 : Construir un modelo de inteligencia artificial capaz de clasificar noticias de prensa según su categoría temática principal.

category: sociedad, salud, politica, medioambiente, internacional, entretenimiento, economia, deportes, cultura, cienciatecnologia


### Primero: cargamos la data que servira para el entrenamiento

In [8]:
#!pip install spacy-lookups-data

import pandas as pd

# Lee el archivo CSV en un DataFrame de pandas

df = pd.read_csv("train_data.csv",thousands=',')

# Obtén 1000 filas aleatorias del DataFrame
random_rows = df.sample(n=1000, random_state=42)

# Encuentra las clases únicas en la columna "clase"
unique_classes = random_rows["clase"].unique()

unique_classes


array(['entretenimiento', 'tecnologia', 'medio_ambiente', 'ciencia',
       'politica', 'internacional', 'accidentes', 'educacion', 'economia',
       'salud', 'deportes'], dtype=object)

In [3]:
# Crea la lista de tuplas con las etiquetas de clases
data_list = []
for _, row in random_rows.iterrows():
    title = row["title"]
    clase = row["clase"]
    cats = {cls: 1 if cls == clase else 0 for cls in unique_classes}
    data_list.append((title, {"cats": cats}))

#data_list

### Segundo: Iniciamos entrenamiento con la data anterior

### ejecutar 2 veces lo siguiente

In [9]:
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch
from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL

# Datos de entrenamiento
TRAIN_DATA = data_list

# Inicializar spaCy con el modelo de lenguaje es_core_news_sm
nlp = spacy.load("es_core_news_sm")

train_examples = []

for example in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(example[0]), example[1]))

def get_examples():
    return train_examples

### Tercero: definimos el modelo a crear

In [5]:
model = {
            "@architectures": "spacy.TextCatCNN.v2",
            "exclusive_classes": True,
            "tok2vec": DEFAULT_TOK2VEC_MODEL,
        }

# Crear un componente TextCategorizer con un modelo CNN
textcat = nlp.add_pipe("textcat", config={"model": model})

textcat.initialize(get_examples)

### Cuanto : Comenzamos a entrenar el modelo

In [6]:
from spacy.util import compounding

# Entrenar el modelo
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.begin_training()
    for epoch in range(100):
        losses = {}
        random.shuffle(TRAIN_DATA)
        # Dividir los datos en lotes y actualizar el modelo
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            texts, annotations = zip(*batch)
            example = []
            # Actualizar el modelo con iteraciones
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
        #print(losses)

### Quinto : Guardamos el modelo 

In [ ]:
# Guardar el modelo entrenado en disco
nlp.to_disk("modelo1")

# Cargar el modelo entrenado
nlp_loaded = spacy.load("modelo1")

#modelo_clasificador_noticias")
